In [1]:
from osgeo import gdal

In [2]:
print(gdal.__version__)

3.11.0dev-b51894d2b5c27a9f9a3b4494937ed5c2c26c384d


In [4]:
raster_path = "/home/data/EEA10_N38_E022.tif"

dataset = gdal.Open(raster_path)

if dataset:
    print("GDAL opened the file successfully.")
    print(f"Raster Size: {dataset.RasterXSize} x {dataset.RasterYSize}")
else:
    print("Failed to open the file.")


GDAL opened the file successfully.
Raster Size: 9001 x 9001


In [6]:
import os
from glob import glob
import rasterio
import numpy as np
# import dask.array as da
# from rasterio.enums import Resampling
# from rasterio.io import MemoryFile
from rasterio.merge import merge


from rasterio import mask

In [7]:
# Path to the directory containing the GeoTIFF files
input_dir = '/home/data/'
output_dir = '/home/data/masks/'
output_mosaic_path = '/home/data/mosaic/'

In [8]:
# Get list of all DEM files (GeoTIFF patches)
dem_files = glob(os.path.join(input_dir, '*.tif'))

In [9]:
# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_mosaic_path, exist_ok=True)

In [10]:
# Function to process each DEM patch and create a binary mask
def process_patch(dem_file):
    with rasterio.open(dem_file) as src:
        # Read the DEM data into a numpy array
        dem_array = src.read(1)
        
        # Create binary mask: 1 for values < 0, else 0
        binary_mask = np.where(dem_array <= 0, 1, 0)
        
        # Define metadata for the binary mask
        meta = src.meta
        meta.update(dtype=rasterio.uint8, count=1)
        
        # Output path for the binary mask
        output_path = os.path.join(output_dir, os.path.basename(dem_file))
        
        # Write the binary mask as a new GeoTIFF
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(binary_mask, 1)
        
        return output_path, binary_mask, meta

# Function to create a mosaic of all binary masks
def create_mosaic(dem_files):
    # List to hold the opened sources
    src_files_to_mosaic = []
    
    for dem_file in dem_files:
        src = rasterio.open(dem_file)
        # Read the binary mask data into a numpy array
        # mask_array = src.read(1)
        # Append source for merging (this keeps the metadata and transform)
        src_files_to_mosaic.append(src)
    
    # Use rasterio.merge to create the mosaic
    # We need to extract both the data and the transform from each source
    # data_to_merge = [src[1] for src in src_files_to_mosaic]
    # transforms = [src[0].transform for src in src_files_to_mosaic]
    
    # Merge the patches together, with the transforms being adjusted to align
    mosaic_array, mosaic_transform = merge(src_files_to_mosaic, resampling=rasterio.enums.Resampling.nearest)

    # Update metadata to match the mosaic
    mosaic_meta = src_files_to_mosaic[0].meta
    mosaic_meta.update({
        'driver': 'GTiff',
        'count': 1,
        'height': mosaic_array.shape[1],
        'width': mosaic_array.shape[2],
        'dtype': 'uint8',
        'crs': src_files_to_mosaic[0].crs,  # Assuming all patches have the same CRS
        'transform': mosaic_transform,  # Updated transform for the mosaic
        'nodata': 1
    })

    return mosaic_array, mosaic_meta
    
# Process all individual patches and create binary masks
binary_mask_paths = []
for dem_file in dem_files:
    binary_mask_path, _, _ = process_patch(dem_file)
    binary_mask_paths.append(binary_mask_path)

# Now create a mosaic of the binary masks
mosaic_mask, mosaic_meta = create_mosaic(binary_mask_paths)
    
# Save the mosaic binary mask as a GeoTIFF
with rasterio.open(os.path.join(output_mosaic_path, 'mosaic_binary_mask.tif'), 'w', **mosaic_meta) as dst:
    dst.write(mosaic_mask.squeeze(), 1)

print("Processing complete. Binary patches and mosaic saved.")

Processing complete. Binary patches and mosaic saved.
